<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/gan/c1_w4_controllable_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Controllable Generation

### Goals

Explore GAN contrallability approaches using gradients from a classifier. By training a classifier to recognize a relevant feature, you can use it to change the generator's input (z-vectors) to make it generate images with more or less of that feature.

### Packages and Visualization

Download CelebA pretrained models.

In [2]:
!wget -q https://github.com/martin-fabbri/colab-notebooks/raw/master/deeplearning.ai/gan/pretrained/pretrained_celeba.pth
!wget -q https://github.com/martin-fabbri/colab-notebooks/raw/master/deeplearning.ai/gan/pretrained/pretrained_classifier.pth

In [3]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.datasets import CelebA
from torch.utils.data import DataLoader

torch.manual_seed(0)

torch.__version__

'1.7.0+cu101'

In [4]:
def show_tensor_images(image_tensor, num_images=16, size=(3, 64, 64), nrow=3):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=nrow)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

### Generator and Noise

In [5]:
class Generator(nn.Module):
    """
    Generator Class
    Values:
        z_dim: the dimension of the noise vector, a scalar
        im_chan: the number of channels in the images, fitted for the dataset
                 used, a scalar (CelebA is rgb, so 3 is our default)
        hidden_dim: the inner dimension, a scalar
    """

    def __init__(self, z_dim=10, im_chan=3, hidden_dim=64):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        # Build the neural network
        self.gen = nn.Sequential(
            self.make_gen_block(z_dim, hidden_dim * 8),
            self.make_gen_block(hidden_dim * 8, hidden_dim * 4),
            self.make_gen_block(hidden_dim * 4, hidden_dim * 2),
            self.make_gen_block(hidden_dim * 2, hidden_dim),
            self.make_gen_block(hidden_dim, im_chan, kernel_size=4, 
                                final_layer=True),
        )

    def make_gen_block(
        self,
        input_channels,
        output_channels,
        kernel_size=3,
        stride=2,
        final_layer=False,
    ):
        """
        Function to return a sequence of operations corresponding to a generator
        block of DCGAN; a transposed convolution, a batchnorm (except in the 
        final layer), and an activation.
        Parameters:
            input_channels: how many channels the input feature representation 
                            has
            output_channels: how many channels the output feature representation 
                             should have
            kernel_size: the size of each convolutional filter, equivalent to 
                         (kernel_size, kernel_size)
            stride: the stride of the convolution
            final_layer: a boolean, true if it is the final layer and false 
                         otherwise (affects activation and batchnorm)
        """
        if not final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(
                    input_channels, output_channels, kernel_size, stride
                ),
                nn.BatchNorm2d(output_channels),
                nn.ReLU(inplace=True),
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(
                    input_channels, output_channels, kernel_size, stride
                ),
                nn.Tanh(),
            )

    def forward(self, noise):
        """
        Function for completing a forward pass of the generator: Given a noise 
        tensor, returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        """
        x = noise.view(len(noise), self.z_dim, 1, 1)
        return self.gen(x)


def get_noise(n_samples, z_dim, device="cpu"):
    """
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim)
    creates a tensor of that shape filled with random numbers from the normal 
    distribution.
    Parameters:
        n_samples: the number of samples in the batch, a scalar
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    """
    return torch.randn(n_samples, z_dim, device=device)


### Classifier

In [10]:
class Classifier(nn.Module):
    """
    Classifier Class
    Values:
        im_chan = the number of channels tin the images, fitted for the dataset
                  used, a scalar
        n_classes: the total number of classes in the dataset, an intger scalar
        hidden_dim: the inner dimension, a scalar
    """
    def __init__(self, im_chan=3, n_classes=2, hidden_dim=64):
        super(Classifier, self).__init__()
        self.classifier = nn.Sequential(
            self.make_classifier_block(im_chan, hidden_dim),
            self.make_classifier_block(hidden_dim, hidden_dim * 2),
            self.make_classifier_block(
                hidden_dim * 2, 
                hidden_dim * 4, 
                stride=3),
            self.make_classifier_block(
                hidden_dim * 4, 
                n_classes, 
                final_layer=True)
        )

    def make_classifier_block(self, input_channels, output_channels, 
                              kernel_size=4, stride=2, final_layer=False):
        """
        Function to return a sequence of operations corresponding to a 
        classifier block; a convolution, a batchnorm (except in the final 
        layer), and an activation (except in the final layer).
        Parameters:
            input_channels: how many channels the input feature representation 
                            has
            output_channels: how many channels the output feature representation 
                             should have
            kernel_size: the size of each convolutional filter, equivalent to 
                         (kernel_size, kernel_size)
            stride: the stride of the convolution
            final_layer: a boolean, true if it is the final layer and false 
                         otherwise 
        """
        if final_layer:
            return nn.Sequential(
                nn.Conv2d(input_channels, output_channels, kernel_size, stride),
            )
        else:
            return nn.Sequential(
                nn.Conv2d(input_channels, output_channels, kernel_size, stride),
                nn.BatchNorm2d(output_channels),
                nn.LeakyReLU(0.2, inplace=True),
            )

    def forward(self, image):
        """
        Function for completing a forward pass of the classifier: Given an image
        tensor, returns an n_classes-dimension tensor representing fake/real.
        Parameters:
            image: a flattened image tensor with im_chan channels
        """
        class_pred = self.classifier(image)
        return class_pred.view(len(class_pred), -1)


### Specifying Parameters

Before you begin training, we need to specify a few parameters:

- z_dim: the dimension of the noise vector
- batch_size: the number of images per forward/backward pass
- device: the device type

In [7]:
z_dim = 64
batch_size = 128
device = 'cuda'

### Train a Classifier

In [8]:
def train_classifier(filename):
    labels_indices = range(40)
    n_epochs = 3
    display_step = 500
    lr = 0.001
    beta_1 = 0.5
    beta_2 = 0.999
    image_size = 64

    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    dataloader = DataLoader(
        CelebA(".", split="train", download=True, transform=transform),
        batch_size=batch_size,
        shuffle=True
    )
    classifier = Classifier(n_classes=len(len_indices)).to(device)
    class_opt = torch.optim.Adam(
        classifier.parameters(), 
        lr=lr, 
        betas=(beta_1, beta_2)
    )
    criterion = nn.BCEWithLogitsLoss()

    cur_step = 0
    classifier_losses = []

    for epoch in range(n_epochs):
        # Dataloader returns the batches
        for real, labels in tqdm(dataloader):
            real = real.to(device)
            labels = labels[:, label_indices].to(device).float()

            class_opt.zero_grad()
            class_pred = classifier(real)
            class_loss = criterion(class_pred, labels)
            class_loss.backward() # Calculate the gradients
            class_opt.step() # Update the weights
            classifier_losses += [class_loss.item()]

            ## Visualization code ##
            if cur_step % display_step == 0 and cur_step > 0:
                class_mean = sum(classifier_losses[-display_step:]) / display_step
                print(f"Step {cur_step}: Classifier loss: {class_mean}")
                step_bins = 20
                x_axis = sorted([i * step_bins for i in range(len(classifier_losses) // step_bins)] * step_bins)
                sns.lineplot(x_axis, classifier_losses[:len(x_axis)], label="Classifier Loss")
                plt.legend()
                plt.show()
                torch.save({"classifier": classifier.state_dict()}, filename)
            cur_step += 1

# Uncomment the last line to train your own classfier - this line will not work in Coursera.
# If you'd like to do this, you'll have to download it and run it, ideally using a GPU 
# train_classifier("filename")


### Loading the Pretrained Model

You will then load the pretrained generator and classifier using the following code.

In [11]:
# load pretrained generator
gen = Generator(z_dim).to(device)
gen_dict = torch.load(
    "pretrained_celeba.pth",
    map_location=torch.device(device)
)["gen"]
gen.load_state_dict(gen_dict)
gen.eval()

n_classes = 40
classifier = Classifier(n_classes=n_classes).to(device)
class_dict = torch.load(
    "pretrained_classifier.pth", 
    map_location=torch.device(device)
)["classifier"]
classifier.load_state_dict(class_dict)
classifier.eval()
print("Pretrained models loading complete.")

opt = torch.optim.Adam(classifier.parameters(), lr=0.01)

Pretrained models loading complete.


### Training

Now you can start implementing a method for controlling your GAN!

For training, you need to write the code to update the noise to produce more of your desired feature. You do this by performing stochastic gradient ascent. You use stochastic gradient ascent to find the local maxima, as opposed to stochastic gradient descent which finds the local minima. Gradient ascent is gradient descent over the negative of the value being optimized. Their formulas are essentially the same, however, instead of subtracting the weighted value, stochastic gradient ascent adds it; it can be calculated by `new = old + (∇ old * weight)`, where ∇ is the gradient of `old`. You perform stochastic gradient ascent to try and maximize the amount of the feature you want. If you wanted to reduce the amount of the feature, you would perform gradient descent. However, in this assignment you are interested in maximize your feature using gradient ascent, since many features in the dataset are not present much more often than they're present and you are trying to add a feature to the images, not remove.

Given the noise with its gradient already calculated through the classifier, you want to return the new noise vector.

<details>

<summary>
<font size="3" color="green">
<b>Optional hint for <code><font size="4">calculate_updated_noise</font></code></b>
</font>
</summary>

1.   Remember the equation for gradient ascent: `new = old + (∇ old * weight)`.

</details>

In [12]:
def calculate_updated_noise(noise, weight):
    """
    Function to return the noise vectors update with stochastic
    gradient ascent.
    Parameters:
        noise: the current noise vectors. You have already called the backwards 
               function on the target class so you can access the gradient of
               the output class with respect to the noise by using noise.grad
        weight: the scalar amount by which you should weight the noise gradient
    """
    #### START CODE HERE ####
    new_noise = noise + noise.grad * weight
    #### END CODE HERE ####
    return new_noise


In [14]:
# Unit test
opt.zero_grad()
noise = torch.ones(20, 20) * 2
noise.requires_grad_()
fake_classes = (noise ** 2).mean()
fake_classes.backward()
new_noise = calculate_updated_noise(noise, 0.1)
assert type(new_noise) == torch.Tensor
assert tuple(new_noise.shape) == (20, 20)
assert new_noise.max() == 2.0010
assert new_noise.min() == 2.0010
assert torch.isclose(new_noise.sum(), torch.tensor(0.4) + 20 * 20 * 2)
print("Success!")

Success!
